In [1]:
pip install ib_insync

In [1]:
from ib_insync import *

# Utilize an instance of IB
ib = IB()

# Connect to the IB server using await in front of the connectAsync method
try:
    await ib.connectAsync('127.0.0.1', 7497, clientId=1)
    print("Connected to Interactive Brokers")
except RuntimeError as e:
    print(f"Runtime Error: {e}")
except Exception as e:
    print(f"An error occurred: {e}")
# from ib_insync import *

# ib = IB()
# ib.connect('127.0.0.1', 7497, clientId=2)

Connected to Interactive Brokers


In [2]:
contract = Future('ES', '202406', 'CME')

In [3]:
await ib.qualifyContractsAsync(contract)

[Future(conId=551601561, symbol='ES', lastTradeDateOrContractMonth='20240621', multiplier='50', exchange='CME', currency='USD', localSymbol='ESM4', tradingClass='ES')]

In [4]:
print(ib.reqMarketDataType(3))

None


In [48]:
from ib_insync import *

# Assume 'ib' is already connected and is an instance of ib_insync.IB()
# Also assume 'contract' has been created and is a valid ib_insync.Contract object

# Use the asynchronous version of the method with await
try:
    historical_data = await ib.reqHistoricalDataAsync(
        contract, endDateTime='', durationStr='30 D',
        barSizeSetting='1 hour', whatToShow='MIDPOINT', useRTH=True
    )
    
    # Proceed with the historical data
#     for bar in historical_data:
#         print(bar)
except Exception as e:
    print(f"An error occurred: {e}")


In [49]:
import pytz
import pandas as pd

# Assuming 'historical_data' contains the historical data returned by reqHistoricalDataAsync

# Convert timestamps to the desired time zone (e.g., 'US/Eastern')
tz = pytz.timezone('US/Eastern')
for bar in historical_data:
    bar.date = bar.date.astimezone(tz)

# Convert historical data to DataFrame
df = pd.DataFrame([vars(bar) for bar in historical_data])

# Print the DataFrame
print(df)


                         date     open     high      low    close  volume  \
0   2024-03-14 09:30:00-04:00  5240.00  5241.50  5208.75  5209.50    -1.0   
1   2024-03-14 10:00:00-04:00  5209.50  5226.00  5203.00  5216.25    -1.0   
2   2024-03-14 11:00:00-04:00  5216.25  5224.00  5210.25  5220.00    -1.0   
3   2024-03-14 12:00:00-04:00  5220.00  5221.75  5208.00  5208.75    -1.0   
4   2024-03-14 13:00:00-04:00  5208.75  5216.00  5196.75  5215.25    -1.0   
..                        ...      ...      ...      ...      ...     ...   
235 2024-04-25 12:00:00-04:00  5050.25  5068.25  5046.50  5065.50    -1.0   
236 2024-04-25 13:00:00-04:00  5065.50  5086.50  5057.25  5083.25    -1.0   
237 2024-04-25 14:00:00-04:00  5083.25  5085.25  5064.50  5072.75    -1.0   
238 2024-04-25 15:00:00-04:00  5072.75  5090.75  5072.00  5082.75    -1.0   
239 2024-04-25 16:00:00-04:00  5082.75  5131.75  5079.00  5125.25    -1.0   

     average  barCount  
0       -1.0        -1  
1       -1.0        -1  


In [51]:
df = util.df(historical_data)
print(df)
def preprocess_and_save_data(data, filename):
    data.to_csv(filename, index=False)
preprocess_and_save_data(df, 'ib_es500_price_history.csv')

                         date     open     high      low    close  volume  \
0   2024-03-14 09:30:00-04:00  5240.00  5241.50  5208.75  5209.50    -1.0   
1   2024-03-14 10:00:00-04:00  5209.50  5226.00  5203.00  5216.25    -1.0   
2   2024-03-14 11:00:00-04:00  5216.25  5224.00  5210.25  5220.00    -1.0   
3   2024-03-14 12:00:00-04:00  5220.00  5221.75  5208.00  5208.75    -1.0   
4   2024-03-14 13:00:00-04:00  5208.75  5216.00  5196.75  5215.25    -1.0   
..                        ...      ...      ...      ...      ...     ...   
235 2024-04-25 12:00:00-04:00  5050.25  5068.25  5046.50  5065.50    -1.0   
236 2024-04-25 13:00:00-04:00  5065.50  5086.50  5057.25  5083.25    -1.0   
237 2024-04-25 14:00:00-04:00  5083.25  5085.25  5064.50  5072.75    -1.0   
238 2024-04-25 15:00:00-04:00  5072.75  5090.75  5072.00  5082.75    -1.0   
239 2024-04-25 16:00:00-04:00  5082.75  5131.75  5079.00  5125.25    -1.0   

     average  barCount  
0       -1.0        -1  
1       -1.0        -1  


In [22]:
order = Order(action='SELL', totalQuantity=5, orderType='LMT', lmtPrice=5300)

In [23]:
trade = ib.placeOrder(contract, order)

In [20]:
order = Order(action='BUY', totalQuantity=9, orderType='LMT', lmtPrice=5070)
trade = ib.placeOrder(contract, order)

In [24]:
from ib_insync import *

# Assuming 'ib' is already connected and is an instance of ib_insync.IB()

# Use the asynchronous version of the accountSummary function with await
account_summary = await ib.accountSummaryAsync()

# Find the total cash balance
balance = next((item for item in account_summary if item.tag == 'TotalCashBalance'), None)
print("Account Balance:", balance.value if balance else "No balance found")

Account Balance: 1081472.3617


In [25]:
import asyncio
def buy(tq, ot, lp):
    order = Order(action='BUY', totalQuantity=tq, orderType=ot, lmtPrice=lp)
    trade = ib.placeOrder(contract, order)
def sell(tq, ot, lp):
    order = Order(action='SELL', totalQuantity=tq, orderType=ot, lmtPrice=lp)
    trade = ib.placeOrder(contract, order)  


async def check_total_cash_balance():
    try:
        account_summary = await ib.accountSummaryAsync()
        balance = next((item for item in account_summary if item.tag == 'TotalCashBalance'), None)
        return balance.value  # Assuming 'balance' has a 'value' attribute.
    except Exception as e:
        return f"An error occurred: {e}"

# Async function to call and print the result of check_total_cash_balance
async def main():
    balance = await check_total_cash_balance()
    print(balance)

# Running the main function
if __name__ == "__main__":
#     asyncio.run(main())
    await main()


1081472.3617


In [58]:
pip install alpaca-trade-api

  Obtaining dependency information for alpaca-trade-api from https://files.pythonhosted.org/packages/0b/b2/4557d0a4c837b020bc5c8971e8fde8b976e332d5c225476699e0b5e30b41/alpaca_trade_api-3.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for websockets<11,>=9.0 from https://files.pythonhosted.org/packages/47/4d/f2e28f112302d3bc794b74ae64656255161d8223f4d47bd17d40cbb3629e/websockets-10.4-cp39-cp39-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for msgpack==1.0.3 from https://files.pythonhosted.org/packages/3a/57/c90cb0b1ab68650ff0068240449bc17c269069bd898cf283dc8db72e8788/msgpack-1.0.3-cp39-cp39-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for aiohttp<4,>=3.8.3 from https://files.pythonhosted.org/packages/20/1c/b7e668a1583cbb1e5060b45baee68a50123f66470ac463f81f7e3b3c3bab/aiohttp-3.9.3-cp39-cp39-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for PyYAML==6.0.1 from https://files.pythonhosted.org/packages/57/c5/5d0


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [61]:
order = Order(action='BUY', totalQuantity=100, orderType='LMT', lmtPrice=5200)
trade = ib.placeOrder(contract, order)

ConnectionError: Not connected

In [1]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [17]:
import alpaca_trade_api as tradeapi

# Set your API key and secret
API_KEY = 'PKHIU3GSGREXWZOJSIP2'
API_SECRET = 'Gi1lOxqYrvvLb3PMNEMEnqa4b0IRTbG73GZEbWVf'
BASE_URL = 'https://paper-api.alpaca.markets'  # or 'https://api.alpaca.markets' for live trading

# Initialize the API connection
api = tradeapi.REST(API_KEY, API_SECRET, base_url=BASE_URL, api_version='v2')

# Get stock data
barset = api.get_bars('ES', '1D', limit=10)
# aapl_bars = barset['AAPL']
for bar in barset:
    print(bar.t, bar.o, bar.h, bar.l, bar.c, bar.v)
    print("close price", bar.c)

2024-04-12 00:00:00-04:00 59.35 59.495 58.43 58.79 2204619
close price 58.79


In [18]:
from datetime import datetime, timedelta

try:
    # Calculate the start and end dates for the last 5 days
    end_date = datetime.today() - timedelta(days=1)  # Fetch data until yesterday
    start_date = end_date - timedelta(days=40)  # Fetch data from 5 days ago

    # Format dates in ISO 8601 format (YYYY-MM-DD)
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')

    # Define the symbols, timeframe, and limit
    symbols = ['AAPL']  # Add more symbols as needed
    timeframe = '1D'  # Daily bars
    limit = 40  # Number of bars to fetch, fetch data for the last 5 days

    # Create an empty dictionary to store historical data for each symbol
    historical_data = {}

    # Fetch historical data for each symbol
    for symbol in symbols:
        barset = api.get_bars(symbol, timeframe, start=start_date_str, end=end_date_str, limit=limit)

        # Store the BarSet object in the dictionary
        historical_data[symbol] = barset

    # Print the fetched data for each symbol
    for symbol, barset in historical_data.items():
        print(f"Data for {symbol}:")
        for bar in barset:
            print("Timestamp:", bar.t)
            print("Open:", bar.o)
            print("High:", bar.h)
            print("Low:", bar.l)
            print("Close:", bar.c)
            print("Volume:", bar.v)
            print()  # Add a newline for clarity between bars
        print()  # Add an extra newline for clarity between symbols

except Exception as e:
    print("Error:", e)

Data for AAPL:
Timestamp: 2024-03-04 00:00:00-05:00
Open: 176.15
High: 176.9
Low: 173.79
Close: 175.1
Volume: 81510101

Timestamp: 2024-03-05 00:00:00-05:00
Open: 170.76
High: 172.04
Low: 169.62
Close: 170.12
Volume: 95432355

Timestamp: 2024-03-06 00:00:00-05:00
Open: 171.06
High: 171.24
Low: 168.68
Close: 169.12
Volume: 68587707

Timestamp: 2024-03-07 00:00:00-05:00
Open: 169.15
High: 170.73
Low: 168.49
Close: 169
Volume: 71765061

Timestamp: 2024-03-08 00:00:00-05:00
Open: 169
High: 173.7
Low: 168.94
Close: 170.73
Volume: 76159346

Timestamp: 2024-03-11 00:00:00-04:00
Open: 172.94
High: 174.38
Low: 172.05
Close: 172.75
Volume: 60139473

Timestamp: 2024-03-12 00:00:00-04:00
Open: 173.15
High: 174.03
Low: 171.01
Close: 173.23
Volume: 59825372

Timestamp: 2024-03-13 00:00:00-04:00
Open: 172.77
High: 173.185
Low: 170.76
Close: 171.13
Volume: 52488692

Timestamp: 2024-03-14 00:00:00-04:00
Open: 172.91
High: 174.3078
Low: 172.05
Close: 173
Volume: 72913507

Timestamp: 2024-03-15 00:00:00-

In [8]:
pip install alpha_vantage

  Obtaining dependency information for alpha_vantage from https://files.pythonhosted.org/packages/ba/b4/d95f9e0eccea6732bab5a079772d453a4f0b68a9f63740d9cf320f92beaa/alpha_vantage-2.3.1-py3-none-any.whl.metadata

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [50]:
from alpha_vantage.timeseries import TimeSeries

# Set your API key
API_KEY = 'your_api_key'

# Initialize the API connection
ts = TimeSeries(key=API_KEY, output_format='pandas')

# Get stock data
data, metadata = ts.get_daily(symbol='ES', outputsize='compact')
print(data)

            1. open  2. high  3. low  4. close  5. volume
date                                                     
2024-04-01    59.71   59.710  58.525     58.88  1503051.0
2024-03-28    59.27   59.985  58.800     59.77  1921036.0
2024-03-27    56.91   58.990  56.910     58.98  2840904.0
2024-03-26    57.49   57.820  56.160     56.29  2165646.0
2024-03-25    57.89   58.250  57.190     57.52  1923841.0
...             ...      ...     ...       ...        ...
2023-11-10    54.57   54.750  53.625     53.80  2398277.0
2023-11-09    55.97   56.130  54.210     54.24  2427103.0
2023-11-08    56.17   56.340  54.900     55.69  2249165.0
2023-11-07    56.37   57.640  55.630     56.35  3137947.0
2023-11-06    54.83   56.610  54.690     56.56  5270655.0

[100 rows x 5 columns]


In [15]:
import requests
from bs4 import BeautifulSoup

# Set the URL for MarketAux
url = 'https://www.marketaux.com/'

try:
    # Send a request to the website and parse the HTML content
    response = requests.get(url)
    response.raise_for_status()  # Raise an error for bad status codes
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract financial headlines
    headlines = soup.find_all('h2', class_='entry-title')
    for headline in headlines:
        print(headline.text)
        
except Exception as e:
    print("An error occurred:", e)

In [15]:
import requests
from dateutil.parser import parse

def get_headlines():
    url = "https://api.marketaux.com/v1/news/all?symbols=^GSPC&filter_entities=true&language=en&api_token="
    
    cleaned_data = []
    for _ in range(3):
        response = requests.get(url)
        news_headlines = response.json()
        print(news_headlines)  # Print the response content
        for article in news_headlines['data']:
            date = article['published_at']
            title = article['title']
            cleaned_data.append({'date': date, 'headline': title})
    
    return cleaned_data

def extract_headlines(data):
    # Initialize an empty dictionary to hold the headlines
    headlines = {}
    
    # Loop through each article in the data file
    for article in data['articles']:
        # Extract the date from the 'publishedAt' field and convert it to a datetime object
        date = parse(article['publishedAt']).date()
        
        # Convert the date to a string in the format 'YYYY-MM-DD'
        date_str = date.strftime('%Y-%m-%d')
        
        # If the date is not already in the headlines dictionary, add it with an empty list as the value
        if date_str not in headlines:
            headlines[date_str] = []
        
        # Append the article's title to the list of headlines for this date
        headlines[date_str].append(article['title'])
data = get_headlines()
extract_headlines(data)

{'error': {'code': 'invalid_api_token', 'message': 'An invalid API token was supplied.'}}


KeyError: 'data'

In [6]:
from alpha_vantage.timeseries import TimeSeries

# Set your API key
API_KEY = 'your_api_key'

# Initialize the API connection
ts = TimeSeries(key=API_KEY, output_format='pandas')

# Get stock data
data, metadata = ts.get_daily(symbol='AAPL', outputsize='compact')
for d in metadata:
    
    print(d)

1. Information
2. Symbol
3. Last Refreshed
4. Output Size
5. Time Zone


In [7]:
import csv

# Data to be written to the CSV file
data = [
    ["Name", "Age", "City"],
    ["Alice", 30, "New York"],
    ["Bob", 25, "Los Angeles"],
    ["Charlie", 35, "Chicago"]
]

# Opening the CSV file and writing the data
with open('people.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)

print("CSV file has been created successfully.")


CSV file has been created successfully.


In [8]:
import pandas as pd

# Define the CSV file name
filename = 'people.csv'

# Read the CSV file
df = pd.read_csv(filename)

# Print the DataFrame
print(df)


      Name  Age         City
0    Alice   30     New York
1      Bob   25  Los Angeles
2  Charlie   35      Chicago


Error: name 'api' is not defined


In [35]:
import requests

# Replace 'your_api_key_here' with your actual MarketAux API key
api_key = 'EQIIjldXyt4Jyk9qre8RAdnbXzVyumxGguFnV7Rj'
endpoint = 'https://api.marketaux.com/v1/news/'
params = {
    'api_key': api_key,
    # Add any other required parameters according to the API documentation
}

try:
    # Make the request to the MarketAux API
    response = requests.get(endpoint, params=params)
    response.raise_for_status()  # Raise an error for bad status codes

    # Parse the JSON response
    data = response.json()

    # Assuming the JSON response contains a list of headlines under 'data' key
    for headline in data.get('data', []):
        print(headline['title'])  # Adjust the key according to the actual response structure
        
#     response = requests.get(endpoint, params=params)
#     response.raise_for_status()  # Raise an error for bad status codes
#     soup = BeautifulSoup(response.content, 'html.parser')

#     # Extract financial headlines
#     # Note: The 'h2' and 'entry-title' class are assumptions. 
#     # Please verify and adjust according to the website's current HTML structure.
#     headlines = soup.find_all('h2', class_='entry-title')
    
except requests.RequestException as e:
    print(f"Request error: {e}")
except ValueError as e:
    print(f"JSON decode error: {e}")



Request error: 404 Client Error: Not Found for url: https://api.marketaux.com/v1/news/?api_key=EQIIjldXyt4Jyk9qre8RAdnbXzVyumxGguFnV7Rj


In [37]:
import requests
from urllib.parse import urlencode

languages = ["en"]
doneglyphs = " ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789.,-"
API_TOKEN = "EQIIjldXyt4Jyk9qre8RAdnbXzVyumxGguFnV7Rj"

def get_a_page_of_headlines(page):
    params = urlencode({"api_token": API_TOKEN, "languages": ",".join(languages), "page": page})
    url = f"https://api.marketaux.com/v1/news/all?{params}"
    r = requests.get(url)
    r.raise_for_status()
    json = r.json()
    return ([x.get("title") for x in json["data"]], json["meta"]["found"] / json["meta"]["returned"])

def filter_nicely(headline):
    # We could make exceptions for special cases here - check if we have
    # punctuation in our supported list and drop punctuation if not, etc.
    # But for now, the simple "do we have all the characters in the headline?"
    if all((g in doneglyphs) for g in headline):
        return headline

page = 1
while True:
    headlines, max_pages = get_a_page_of_headlines(page)
    for headline in headlines:
        if filter_nicely(headline):
            print(headline)
    page += 1
    if page > max_pages:
        break


Digital Identity Solutions Provider Signicat Acquires SmartWorks
Ontario Newsroom
Alkhaleej Training board proposes withholding dividends for 2023
STMicroelectronics NV at Citi TMT Conference Transcript
CaixaBank SA Annual Shareholders Meeting Transcript
Kijiji Scammers and General Idiots
Securitas AB Investor Day Transcript
Our Economy and Politics Are Broken
Full Year 2023 Wendel SE Earnings Call Transcript
Side hustles transforming into profitable ventures
End of Day Message
Word-Of-Mouth Is Still How To Get Customers
East Pipes inks SAR 230M supply contract with SWCC
Modern Mills to make debut on TASI on March 27
RBL Bank aims to grow retail with new products, cut back on wholesale book
Jefferies raises LVMH target to EUR790 on steady market share By Investing.com
AXA SA at Morgan Stanley European Financials Conference Transcript
Quinta Brunson To Receive Honorary Degree From Temple University
Rising Interest Costs Squeeze Junk-Rated US Companies Amid Fed Rate Hikes
20 Best Countri

HTTPError: 402 Client Error: Payment Required for url: https://api.marketaux.com/v1/news/all?api_token=EQIIjldXyt4Jyk9qre8RAdnbXzVyumxGguFnV7Rj&languages=en&page=101

In [4]:
pip install tweepy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 kB 1.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 12.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 5.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 6.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.4/426.4 kB 7.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 12.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 4.4 MB/s eta 0:00:00ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [19]:
from ib_insync import *
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import asyncio
import tweepy
from transformers import pipeline
from alpha_vantage.timeseries import TimeSeries
from alpaca_trade_api.rest import REST as AlpacaAPI

# Twitter API for sentiment analysis
api_key = "your_twitter_api_key"
api_key_secret = "your_twitter_api_key_secret"
access_token = "your_access_token"
access_token_secret = "your_access_token_secret"

# Alpaca and Alpha Vantage API keys
ALPACA_API_KEY = 'your_alpaca_api_key'
ALPACA_SECRET_KEY = 'your_alpaca_secret_key'
ALPACA_ENDPOINT = 'https://paper-api.alpaca.markets'
ALPHA_VANTAGE_API_KEY = 'your_alpha_vantage_api_key'

# Initialize Twitter API
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
twitter_api = tweepy.API(auth)

# Initialize Alpaca API
alpaca_api = AlpacaAPI(ALPACA_API_KEY, ALPACA_SECRET_KEY, base_url=ALPACA_ENDPOINT)

def fetch_tweets_about_sp500():
    tweets = twitter_api.search_tweets(q="S&P 500 OR SP500", lang="en", count=100)
    return [tweet.text for tweet in tweets]

def analyze_sentiment(tweets):
    sentiment_pipeline = pipeline("sentiment-analysis")
    results = sentiment_pipeline(tweets)
    sentiment_score = sum([1 if res['label'] == 'POSITIVE' else -1 for res in results]) / len(results)
    return sentiment_score

def fetch_market_data_alpaca(symbol, start_date, end_date):
    barset = alpaca_api.get_barset(symbol, 'day', start=start_date, end=end_date)
    data = barset[symbol]
    df = pd.DataFrame({
        'time': [bar.t for bar in data],
        'open': [bar.o for bar in data],
        'high': [bar.h for bar in data],
        'low': [bar.l for bar in data],
        'close': [bar.c for bar in data],
        'volume': [bar.v for bar in data]
    })
    return df

def preprocess_and_save_data(data, filename='preprocessed_data.csv'):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data['close'].values.reshape(-1,1))
    preprocessed_df = pd.DataFrame(scaled_data, columns=['Scaled_Close'])
    preprocessed_df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")
    return scaled_data

def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(units=50))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def place_order(ib, action, quantity, price=None):
    contract = Future('ES', '202406', 'GLOBEX')
    order = MarketOrder(action, quantity) if not price else LimitOrder(action, quantity, price)
    trade = ib.placeOrder(contract, order)
    print(f"Placed {action} order for {quantity} units.")

async def main():
    ib = IB()
    await ib.connectAsync('127.0.0.1', 7497, clientId=1)
    
    df = fetch_market_data_alpaca('SPY', '2023-01-01', '2023-12-31')
    preprocessed_data = preprocess_and_save_data(df, 'market_data_preprocessed.csv')
    
    tweets = fetch_tweets_about_sp500()
    sentiment_score = analyze_sentiment(tweets)
    
    if sentiment_score > 0:
        place_order(ib, 'BUY', 1)  # Placeholder logic for demonstration
    else:
        place_order(ib, 'SELL', 1)  # Placeholder logic for demonstration

if __name__ == "__main__":
#     asyncio.run(main())
    print(fetch_tweets_about_sp500())
    print('ok')

C:\Users\15596\anaconda3\lib\site-packages\scipy\__init__.py:94: RuntimeWarning: coroutine 'main' was never awaited
  _fun = _deprecated(_msg.format(_key))(_fun)
C:\Users\15596\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Unauthorized: 401 Unauthorized
89 - Invalid or expired token.

In [30]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import tweepy
from transformers import pipeline
from alpaca_trade_api.rest import REST, TimeFrame
from datetime import datetime, timedelta


# Calculate the start and end dates for the last 5 days
end_date = datetime.today() - timedelta(days=1)  # Fetch data until yesterday
start_date = end_date - timedelta(days=365)  # Fetch data from 5 days ago

# Format dates in ISO 8601 format (YYYY-MM-DD)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')
    
# Initialize Alpaca API
ALPACA_API_KEY = 'PKHIU3GSGREXWZOJSIP2'
ALPACA_SECRET_KEY = 'Gi1lOxqYrvvLb3PMNEMEnqa4b0IRTbG73GZEbWVf'
alpaca_api = REST(ALPACA_API_KEY, ALPACA_SECRET_KEY, base_url='https://paper-api.alpaca.markets')

# Initialize Twitter API
auth = tweepy.OAuthHandler("FOzAGryMn6KTGhZ4oCPXZrwDw", "D9EUFQU5x9yk47UVPipGltuokzVeE3JZ9fScogcNmwHaRgrOwF")
auth.set_access_token("1773744788291960837-2eaPvcMt0ADfRsxSMNy77QN4kjb874", "uiPMWeX1UVOa7qRYylvaRXaEqRQ8qIgQgBjvkSLGdleim")
twitter_api = tweepy.API(auth)

# Initialize Transformers Pipeline for Sentiment Analysis

# sentiment_pipeline = pipeline("sentiment-analysis")

def fetch_market_data(symbol, start_date, end_date):
    data = alpaca_api.get_bars(symbol, TimeFrame.Day, start_date, end_date).df
    return data

# def fetch_tweets_about_sp500(count=1):
#     tweets = twitter_api.search_tweets(q="S&P 500 OR SP500", lang="en", result_type="recent", count=count)
#     return [tweet.text for tweet in tweets]

def analyze_sentiment(tweets):
    results = sentiment_pipeline(tweets)
    scores = [1 if res['label'] == 'POSITIVE' else -1 for res in results]
    return np.mean(scores)

def preprocess_data(market_data, sentiment_score):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(market_data.values.reshape(-1, 1))
    # Include sentiment score as a feature
    features = np.hstack((scaled_data, np.full((scaled_data.shape[0], 1), fill_value=sentiment_score)))
    return features[:-1], scaled_data[1:]  # X, y for model training

def preprocess_for_lstm(market_data, sentiments):
    # Assuming market_data is a DataFrame with a 'close' column
    # and sentiments is a list of sentiment scores (-1 for negative, 0 for neutral, 1 for positive)
    market_data['sentiment'] = sentiments[:len(market_data)]  # Truncate or pad sentiments to match market_data length
    features = market_data[['close', 'sentiment']].values
    return features

def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(units=50))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def analyze_sentiment_with_bert(news):
    sentiment_pipeline = pipeline("sentiment-analysis")
    return sentiment_pipeline(news)

def preprocess_and_save_data(data, filename):
#     scaler = MinMaxScaler(feature_range=(0, 1))
#     scaled_data = scaler.fit_transform(data.values.reshape(-1,1))
#     preprocessed_df = pd.DataFrame(scaled_data, columns=['Timestamp', 'Close','High','Low','Trade_count','Open', 'Volume',' Volume Weighted Average Price'])
#     preprocessed_df.to_csv(filename, index=False)
#     print(f"Data saved to {filename}")
#     return scaled_data
    data.to_csv(filename, index=False)
#     return scaled_data

def main():
    print(fetch_market_data('ES', start_date_str, end_date_str))
    market_data = fetch_market_data('ES', start_date_str, end_date_str)
#     tweets = fetch_tweets_about_sp500()
#     sentiment_score = analyze_sentiment(tweets)
#     X, y = preprocess_data(market_data, sentiment_score)
    
#     model = create_lstm_model((X.shape[1], X.shape[2]))
#     model.fit(X, y, epochs=10, batch_size=32)

    print(preprocess_and_save_data(market_data, 'es500_price_history.csv'))

    # Simplified decision making based on the last day's prediction
#     last_sequence = X[-1].reshape(1, X.shape[1], X.shape[2])
#     predicted_change = model.predict(last_sequence)
#     decision = "BUY" if predicted_change > 0 else "SELL"
#     print(f"Trading decision based on model prediction: {decision}")

if __name__ == "__main__":
    main()


                           close     high    low  trade_count   open   volume  \
timestamp                                                                       
2023-04-12 04:00:00+00:00  79.77  80.4800  79.49        23716  80.41  1868297   
2023-04-13 04:00:00+00:00  79.62  79.9300  78.12        17196  79.25  3175246   
2023-04-14 04:00:00+00:00  78.46  78.9900  78.10        25264  78.66  1676812   
2023-04-17 04:00:00+00:00  79.14  79.4705  78.29        18519  78.81  1540053   
2023-04-18 04:00:00+00:00  78.38  79.1500  78.04        18997  78.94  1629106   
...                          ...      ...    ...          ...    ...      ...   
2024-04-05 04:00:00+00:00  58.51  58.8488  58.06        20408  58.55  1426325   
2024-04-08 04:00:00+00:00  59.47  59.6100  58.58        22551  58.58  1654333   
2024-04-09 04:00:00+00:00  59.96  60.4400  59.43        25039  59.57  1862118   
2024-04-10 04:00:00+00:00  59.07  59.3900  58.28        31914  59.24  2344801   
2024-04-11 04:00:00+00:00  5

In [20]:
#use only LSTM model and past past history to predict price and make trading decision
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Step 1: Load and preprocess the data
def load_and_preprocess_data(filepath):
    data = pd.read_csv(filepath)
    close_prices = data['close'].values.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_close_prices = scaler.fit_transform(close_prices)
    return scaled_close_prices, scaler

# Create sequences from the data
def create_sequences(data, sequence_length=60):
    xs, ys = [], []
    for i in range(len(data) - sequence_length):
        x = data[i:(i + sequence_length)]
        y = data[i + sequence_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Step 2: Define the LSTM model
def build_model(input_shape):
    model = Sequential([
        LSTM(units=50, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(units=50, return_sequences=False),
        Dropout(0.2),
        Dense(units=25),
        Dense(units=1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Step 3: Train the model
def train_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Step 4: Make a trading decision
def make_trading_decision(model, latest_sequence_scaled, scaler):
    predicted_price_scaled = model.predict(latest_sequence_scaled[np.newaxis, :, :])
    predicted_price = scaler.inverse_transform(predicted_price_scaled)[0, 0]
    latest_known_price = scaler.inverse_transform(latest_sequence_scaled[-1].reshape(-1, 1))[0, 0]
    
    if predicted_price > latest_known_price:
        decision = "BUY"
    elif predicted_price < latest_known_price:
        decision = "SELL"
    else:
        decision = "HOLD"
    
    return decision

if __name__ == "__main__":
    filepath = 'es500_price_history.csv'  # Replace this with the path to your CSV file
    scaled_close_prices, scaler = load_and_preprocess_data(filepath)
    X, y = create_sequences(scaled_close_prices)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = build_model((X_train.shape[1], 1))
    train_model(model, X_train, y_train, X_test, y_test)
    
    # Example: Making a trading decision based on the latest sequence of prices
    latest_sequence_scaled = X_test[-1]  # Assuming this is the latest available sequence
    decision = make_trading_decision(model, latest_sequence_scaled, scaler)
    print(f"Trading decision: {decision}")


Epoch 1/100


C:\Users\15596\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - loss: 0.1236 - val_loss: 0.0269
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0218 - val_loss: 0.0299
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0199 - val_loss: 0.0204
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0137 - val_loss: 0.0228
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0110 - val_loss: 0.0168
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0080 - val_loss: 0.0151
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0106 - val_loss: 0.0133
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0069 - val_loss: 0.0133
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0080 - val_loss: 0.0123
Epoch 10/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0064 - val_loss: 0.0110
Epoch 11/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0071 - val_loss: 0.0106
Epoch 12/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0065 - val_loss: 0.0107
E

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0039 - val_loss: 0.0052
Epoch 71/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0039 - val_loss: 0.0049
Epoch 72/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0039 - val_loss: 0.0053
Epoch 73/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0046 - val_loss: 0.0051
Epoch 74/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0035 - val_loss: 0.0054
Epoch 75/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0041 - val_loss: 0.0049
Epoch 76/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0039 - val_loss: 0.0048
Epoch 77/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0040 - val_loss: 0.0051
Epoch 78/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0036 - val_loss: 0.0047
Epoch 79/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0037 - val_loss: 0.0048
Epoch 80/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0044 - val_loss: 0.0051
Epoch 81/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0034 - val_loss: 

In [21]:
#use only LSTM model and past past history to predict price and make trading decision, with decision to hold added
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Load and preprocess data
def load_and_preprocess_data(filepath):
    data = pd.read_csv(filepath)
    close_prices = data['close'].values.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_close_prices = scaler.fit_transform(close_prices)
    return scaled_close_prices, scaler

# Create sequences
def create_sequences(data, sequence_length=60):
    xs, ys = [], []
    for i in range(len(data)-sequence_length):
        x = data[i:(i+sequence_length)]
        y = data[i+sequence_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Define LSTM model
def build_model(input_shape):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(25),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Predict and decide
def predict_and_decide(current_price, recent_prices, model, scaler):
#     recent_prices = np.append(recent_prices, current_price).reshape(-1, 1)
#     scaled_recent_prices = scaler.transform(recent_prices)
    
#     # Adjust for the sequence length expected by the model
#     if len(scaled_recent_prices) > 60:
#         scaled_recent_prices = scaled_recent_prices[-60:]
#     elif len(scaled_recent_prices) < 60:
#         scaled_recent_prices = np.pad(scaled_recent_prices, (60-len(scaled_recent_prices), 0), 'constant', constant_values=(0))
    
#     predicted_price_scaled = model.predict(scaled_recent_prices.T[np.newaxis, :, :])
#     predicted_price = scaler.inverse_transform(predicted_price_scaled)[0, 0]
    
#     decision = "BUY" if predicted_price > current_price else "SELL" if predicted_price < current_price else "HOLD"
#     return decision

    # Ensure recent_prices includes the current_price and has the correct shape
    recent_prices = np.append(recent_prices[-59:], current_price).reshape(-1, 1)
    scaled_recent_prices = scaler.transform(recent_prices)
    
    # Reshape for the model: (1, sequence_length, 1)
    scaled_recent_prices = scaled_recent_prices.reshape(1, -1, 1)
    
    # Predict the next day's price
    predicted_price_scaled = model.predict(scaled_recent_prices)
    predicted_price = scaler.inverse_transform(predicted_price_scaled)[0, 0]
    
    # Make a decision
    decision = "BUY" if predicted_price > current_price else "SELL" if predicted_price < current_price else "HOLD"
    return decision


if __name__ == "__main__":
    # Load and preprocess data
    filepath = 'es500_price_history.csv'  # Update with the path to your data
    scaled_close_prices, scaler = load_and_preprocess_data(filepath)
    X, y = create_sequences(scaled_close_prices)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Build and train model
    model = build_model((X_train.shape[1], 1))
    model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1)
    
    # Assuming you have a list/array of recent prices and the current price
    recent_prices = scaled_close_prices[-59:]  # Get the last 59 prices
    current_price = 78.05  # This would be dynamically obtained in a real scenario
    
    # Make decision
    decision = predict_and_decide(current_price, recent_prices, model, scaler)
    print(f"Decision based on current price {current_price}: {decision}")


Epoch 1/100


C:\Users\15596\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - loss: 0.0936 - val_loss: 0.0115
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0202 - val_loss: 0.0107
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0120 - val_loss: 0.0145
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0128 - val_loss: 0.0125
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0112 - val_loss: 0.0068
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0085 - val_loss: 0.0061
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0082 - val_loss: 0.0081
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0071 - val_loss: 0.0076
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0067 - val_loss: 0.0054
Epoch 10/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0061 - val_loss: 0.0055
Epoch 11/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0076 - val_loss: 0.0074
Epoch 12/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0058 - val_loss: 0.0047
E

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0037 - val_loss: 0.0029
Epoch 71/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0039 - val_loss: 0.0020
Epoch 72/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0035 - val_loss: 0.0027
Epoch 73/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0038 - val_loss: 0.0033
Epoch 74/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0040 - val_loss: 0.0023
Epoch 75/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0035 - val_loss: 0.0025
Epoch 76/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0041 - val_loss: 0.0027
Epoch 77/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0052 - val_loss: 0.0024
Epoch 78/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0040 - val_loss: 0.0017
Epoch 79/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0041 - val_loss: 0.0028
Epoch 80/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0038 - val_loss: 0.0023
Epoch 81/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0036 - val_loss: 

In [22]:
import alpaca_trade_api as tradeapi

# Set your API key and secret
API_KEY = 'PKHIU3GSGREXWZOJSIP2'
API_SECRET = 'Gi1lOxqYrvvLb3PMNEMEnqa4b0IRTbG73GZEbWVf'
BASE_URL = 'https://paper-api.alpaca.markets'  # or 'https://api.alpaca.markets' for live trading

# Initialize the API connection
api = tradeapi.REST(API_KEY, API_SECRET, base_url=BASE_URL, api_version='v2')

# Get stock data
barset = api.get_bars('MES', '1D', limit=1)
# aapl_bars = barset['ES']
for bar in barset:
    print(bar.t, bar.o, bar.h, bar.l, bar.c, bar.v)
    current_price = bar.c
    print("close price", bar.c)
    
# decision = predict_and_decide(current_price, recent_prices, model, scaler)
# print(f"Decision based on current price {current_price}: {decision}")

In [23]:
#sentiment analysis using fetched marketaux headline
import requests
from urllib.parse import urlencode

API_TOKEN = "EQIIjldXyt4Jyk9qre8RAdnbXzVyumxGguFnV7Rj"
languages = ["en"]

def get_sp500_sentiment():
    params = {
        "api_token": API_TOKEN,
        "languages": ",".join(languages),
        "query": "S&P 500",  # Assuming 'query' parameter can be used to filter news by keywords
    }
    url = f"https://api.marketaux.com/v1/sentiment?{urlencode(params)}"  # Hypothetical endpoint for sentiment
    response = requests.get(url)
    response.raise_for_status()
    json_response = response.json()
    
    # Assuming the API response contains a sentiment score for the query
    sentiment_score = json_response.get("sentiment_score")
    return sentiment_score

# Fetch and print the sentiment score for S&P 500-related news
sp500_sentiment_score = get_sp500_sentiment()
print(f"S&P 500 Sentiment Score: {sp500_sentiment_score}")

HTTPError: 404 Client Error: Not Found for url: https://api.marketaux.com/v1/sentiment?api_token=EQIIjldXyt4Jyk9qre8RAdnbXzVyumxGguFnV7Rj&languages=en&query=S%26P+500

In [26]:
from ib_insync import *

# Assuming 'ib' is already connected and is an instance of ib_insync.IB()

# Use the asynchronous version of the accountSummary function with await
account_summary = await ib.accountSummaryAsync()

# Find the total cash balance
balance = next((item for item in account_summary if item.tag == 'TotalCashBalance'), None)
print("Account Balance:", balance.value if balance else "No balance found")
available_funds = balance.value
decision, current_profit = predict_and_decide(current_price, recent_prices, model, scaler, available_funds, current_shares, total_cost)
print(f"Decision: {decision[0]} | Shares: {decision[1]} | Current Profit: ${current_profit:.2f}")



Account Balance: 1047515.017
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Decision: SELL | Shares: 50 | Current Profit: $202.50


In [9]:
def fetch_positions():
    positions = ib.positions()
    for pos in positions:
        if pos.contract.symbol == 'ES' and pos.contract.secType == 'FUT':
            print(f"Contract: {pos.contract.localSymbol} | Shares: {pos.position} | Avg Cost: {pos.avgCost}")

# Fetch and print ES positions
fetch_positions()


Contract: ESM4 | Shares: -4.0 | Avg Cost: 265410.25


In [10]:
!pip install nest_asyncio

In [11]:
import nest_asyncio
nest_asyncio.apply()

def fetch_current_price(ib, contract):
    # Request market data snapshot for the contract
    market_data = ib.reqMktData(contract, '', False, False)
#     ib.sleep(2)  # Wait for the data to be returned
    util.sleep(5)
    if market_data:
        print(f"Current Price of {contract.symbol}: Last - {market_data.last}, Bid - {market_data.bid}, Ask - {market_data.ask}")
    else:
        print("Failed to fetch current price.")
contract = Future('ES', '202406', 'CME')
fetch_current_price(ib, contract)

Current Price of ES: Last - 5084.0, Bid - 5083.75, Ask - 5084.0


In [27]:
#Model updated to learn from the data from IB
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Load and preprocess data
def load_and_preprocess_data(filepath):
    data = pd.read_csv(filepath)
    close_prices = data['close'].values.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_close_prices = scaler.fit_transform(close_prices)
    return scaled_close_prices, scaler

# Create sequences for LSTM training
def create_sequences(data, sequence_length=60):
    xs, ys = [], []
    for i in range(len(data) - sequence_length):
        x = data[i:(i + sequence_length)]
        y = data[i + sequence_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Define LSTM model structure
def build_model(input_shape):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(25),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Predict next day's price and make a decision
def predict_and_decide(current_price, recent_prices, model, scaler, available_funds, current_shares, total_cost, transaction_fee=10.0):
    recent_prices = np.append(recent_prices[-59:], current_price).reshape(-1, 1)
#     print(recent_prices)
    scaled_recent_prices = scaler.transform(recent_prices)
    scaled_recent_prices = scaled_recent_prices.reshape(1, -1, 1)
    print(scaled_recent_prices)
    
    predicted_price_scaled = model.predict(scaled_recent_prices)
    predicted_price = scaler.inverse_transform(predicted_price_scaled)[0, 0]
    print(predicted_price,current_price)
    
    # Decision making with transaction fees considered
    if predicted_price > current_price:
        potential_profit = (predicted_price - current_price) * current_shares
        print(potential_profit)
        if available_funds > transaction_fee and potential_profit > 0:
            # Calculate optimal number of shares to buy based on potential profit and available funds
            # Here we assume a strategy to use a fraction of available funds
            funds_for_use = available_funds * 0.1 - transaction_fee  # Example: using 10% of available funds
            max_affordable_shares = int(funds_for_use / current_price)
            decision = "BUY", max_affordable_shares
        else:
            decision = "HOLD", 0
    elif predicted_price < current_price:
        potential_loss = (current_price - predicted_price) * current_shares
        print(potential_loss)
        if potential_loss > 0:
            # Sell a fraction of shares to minimize potential loss
            shares_to_sell = int(current_shares * 0.1)  # Example: selling 10% of holdings
            decision = "SELL", shares_to_sell
        else:
            decision = "HOLD", 0
    else:
        decision = "HOLD", 0
    
    current_profit = (current_price * current_shares) - total_cost
    return decision, current_profit

if __name__ == "__main__":
    filepath = 'ib_es500_price_history.csv'  # Updated filepath to the uploaded CSV
    scaled_close_prices, scaler = load_and_preprocess_data(filepath)
    X, y = create_sequences(scaled_close_prices)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = build_model((X_train.shape[1], 1))
    model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1)
    
    positions = ib.positions()
    for pos in positions:
        if pos.contract.symbol == 'ES' and pos.contract.secType == 'FUT':
            current_shares = pos.position
    account_summary = await ib.accountSummaryAsync()

    # Find the total cash balance
    balance = next((item for item in account_summary if item.tag == 'TotalCashBalance'), None)
    
    contract = Future('ES', '202406', 'CME')
    market_data = ib.reqMktData(contract, '', False, False)
    util.sleep(5)
    current_price = (market_data.bid + market_data.ask) / 2

    
    available_funds = balance.value
    recent_prices = scaled_close_prices[-60:-1]  # Using the last 59 prices for prediction
    total_cost = current_shares * current_price  # Adjusted total cost calculation
    print(current_price, recent_prices, model, scaler, available_funds, current_shares, total_cost)
    
    decision, current_profit = predict_and_decide(current_price, recent_prices, model, scaler, available_funds, current_shares, total_cost)
    print(f"Decision: {decision[0]} | Shares: {decision[1]} | Current Profit: ${current_profit:.2f}")


Epoch 1/100


C:\Users\15596\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 96ms/step - loss: 0.3589 - val_loss: 0.0430
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0507 - val_loss: 0.0788
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0645 - val_loss: 0.0156
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0254 - val_loss: 0.0223
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0303 - val_loss: 0.0211
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0279 - val_loss: 0.0122
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0165 - val_loss: 0.0177
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0217 - val_loss: 0.0159
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0168 - val_loss: 0.0105
Epoch 10/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0144 - val_loss: 0.0101
Epoch 11/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0173 - val_loss: 0.0091
Epoch 12/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0159 - val_loss: 0.0094
E

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0096 - val_loss: 0.0054
Epoch 71/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0089 - val_loss: 0.0054
Epoch 72/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0097 - val_loss: 0.0053
Epoch 73/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0086 - val_loss: 0.0054
Epoch 74/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0099 - val_loss: 0.0054
Epoch 75/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0115 - val_loss: 0.0052
Epoch 76/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0084 - val_loss: 0.0054
Epoch 77/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0104 - val_loss: 0.0053
Epoch 78/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0105 - val_loss: 0.0052
Epoch 79/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0101 - val_loss: 0.0051
Epoch 80/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0091 - val_loss: 0.0051
Epoch 81/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0119 - val_loss: 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Load and preprocess data
def load_and_preprocess_data(filepath):
    data = pd.read_csv(filepath)
    # Adding more features: high, low, and volume
    features = data[['close', 'high', 'low', 'volume']].values
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_features = scaler.fit_transform(features)
    return scaled_features, scaler

# Create sequences for LSTM training
def create_sequences(data, sequence_length=60):
    xs, ys = [], []
    for i in range(len(data) - sequence_length):
        x = data[i:(i + sequence_length)]
        y = data[i + sequence_length, 0]  # Predicting the next close price
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Define LSTM model structure
def build_model(input_shape):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(50),
        Dropout(0.2),
        Dense(25),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Adjust the prediction function to handle new features
def predict_and_decide(model, scaler, recent_data, current_price, available_funds, current_shares, total_cost, transaction_fee=10.0):
#     if recent_data.shape[0] != 60:
#         raise ValueError("recent_data should contain exactly 60 timesteps")
    
#     # Scale the recent_data if it's not already scaled
#     scaled_recent_data = scaler.transform(recent_data)
#     scaled_recent_data = scaled_recent_data.reshape(1, 60, -1)  # Reshape for LSTM input
#     prediction = model.predict(scaled_recent_data)
#     predicted_price = scaler.inverse_transform(prediction)[0][0]
    # Assuming recent_data is in the correct shape and already scaled appropriately
    scaled_recent_data = recent_data.reshape(1, 60, -1)  # Reshape for LSTM input
    
    prediction_scaled = model.predict(scaled_recent_data)
    
    # Since we're only interested in the 'close' price (first feature), we need to prepare a dummy array for inverse transform
    dummy_features = np.zeros((1, 4))  # Assume 4 features as per your scaler fitting
    dummy_features[0, 0] = prediction_scaled[0][0]  # Place the predicted price in the 'close' position
    
    predicted_price = scaler.inverse_transform(dummy_features)[0, 0]  # Inverse transform and extract the 'close' price
    print(predicted_price, current_price)
    # Decision making with transaction fees considered
    decision = "HOLD"
    shares = 0
    if predicted_price > current_price:
        if available_funds > transaction_fee:
            max_affordable_shares = int((available_funds - transaction_fee) / current_price)
            decision = "BUY"
            shares = max_affordable_shares  # Example: Buy as much as possible within budget
    elif predicted_price < current_price:
        decision = "SELL"
        shares = int(current_shares * 0.1)  # Example: Sell 10% of holdings
    
    return decision, shares, predicted_price

# Example usage of the functions
if __name__ == "__main__":
    filepath = 'ib_es500_price_history.csv'
    scaled_features, scaler = load_and_preprocess_data(filepath)
    X, y = create_sequences(scaled_features)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = build_model((X_train.shape[1], X_train.shape[2]))
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))  # Reduced epochs, added explicit test data for validation
    

    # Simulating a scenario for prediction and decision making
    positions = ib.positions()
    for pos in positions:
        if pos.contract.symbol == 'ES' and pos.contract.secType == 'FUT':
            current_shares = pos.position
    account_summary = await ib.accountSummaryAsync()
    # Find the total cash balance
    balance = next((item for item in account_summary if item.tag == 'TotalCashBalance'), None)
    
    contract = Future('ES', '202406', 'CME')
    market_data = ib.reqMktData(contract, '', False, False)
    util.sleep(5)
    current_price = (market_data.bid + market_data.ask) / 2

    
    available_funds = balance.value
    total_cost = current_shares * current_price  # Simulated total cost of current shares
    
    # Prepare recent data for prediction
    recent_data = scaled_features[-60:]  # Make sure this is exactly 60 timesteps
    decision, shares, predicted_price = predict_and_decide(model, scaler, recent_data, current_price, available_funds, current_shares, total_cost)
    print(f"Decision: {decision}, Shares: {shares}, Predicted Price: {predicted_price}")
    
    # Execute the decision
    if decision == "BUY":
        order = MarketOrder('BUY', shares)
        trade = ib.placeOrder(contract, order)
    elif decision == "SELL":
        order = MarketOrder('SELL', shares)
        trade = ib.placeOrder(contract, order)
    elif decision == "HOLD":
        print("Decision is to HOLD, no action taken.")

    # Monitor the order until it is filled
    ib.sleep(1)  # Sleeping to give time for the order to be executed


    print(f"Order {decision} for {shares} shares has been filled.")


In [ ]:
available_funds = 10000
current_price = 5012
recent_prices = scaled_close_prices[-60:-1]
current_shares = 50
total_cost = current_shares*current_price
decision, current_profit = predict_and_decide(current_price, recent_prices, model, scaler, available_funds, current_shares, total_cost)
print(f"Decision: {decision[0]} | Shares: {decision[1]} | Current Profit: ${current_profit:.2f}")


In [53]:
import asyncio

# Assuming these classes and instances are defined elsewhere in your code
class Order:
    def __init__(self, action, totalQuantity, orderType, lmtPrice):
        self.action = action
        self.totalQuantity = totalQuantity
        self.orderType = orderType
        self.lmtPrice = lmtPrice

# Placeholder for the 'contract' and 'ib' instances
# contract = None  # You should define this properly
# ib = None  # Initialize this with your brokerage interface instance

async def buy(tq, ot, lp):
    order = Order(action='BUY', totalQuantity=tq, orderType=ot, lmtPrice=lp)
    trade = await ib.placeOrder(contract, order)

async def sell(tq, ot, lp):
    order = Order(action='SELL', totalQuantity=tq, orderType=ot, lmtPrice=lp)
    trade = await ib.placeOrder(contract, order)

async def check_total_cash_balance():
    try:
        account_summary = await ib.accountSummaryAsync()
        balance = next((item for item in account_summary if item.tag == 'TotalCashBalance'), None)
        return balance.value  # Assuming 'balance' has a 'value' attribute.
    except Exception as e:
        return f"An error occurred: {e}"

async def main():
    balance = await check_total_cash_balance()
    print(balance)

# Adjusted way to run the main function in environments with an existing event loop
if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    if loop.is_running():
        # In environments like Jupyter, directly schedule the coroutine to be run
        task = loop.create_task(main())
    else:
        # In standard environments without a running loop, use asyncio.run()
        asyncio.run(main())

1096753.8012
